In [1]:
import metaspore as ms

spark_confs={
    "spark.network.timeout":"500",
    "spark.ui.showConsoleProgress": "true",
    "spark.kubernetes.executor.deleteOnTermination":"true",
}
spark = ms.spark.get_session(local=False,
                            app_name='ESMM read data',
                            batch_size=256,
                            worker_count=4,
                            server_count=4,
                            worker_memory='5G',
                            server_memory='5G',
                            coordinator_memory='5G',
                            spark_confs=spark_confs)
sc = spark.sparkContext
print('Debug -- spark init')
print('Debug -- version:', sc.version)   
print('Debug -- applicaitonId:', sc.applicationId)
print('Debug -- uiWebUrl:', sc.uiWebUrl)

22/05/18 02:23:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Debug -- spark init
Debug -- version: 3.1.2
Debug -- applicaitonId: spark-application-1652840612130
Debug -- uiWebUrl: http://jupyter.my.nginx.test/hub/user-redirect/proxy/4040/jobs/


In [2]:
train_path = 's3://dmetasoul-bucket/demo/aliccp/traindata_4000w.csv'
test_path = 's3://dmetasoul-bucket/demo/aliccp/testdata_4000w.csv'

train_output_path ='s3://dmetasoul-bucket/demo/aliccp/traindata_4000w.parquet'
test_output_path = 's3://dmetasoul-bucket/demo/aliccp/testdata_4000w.parquet'

In [15]:
from pyspark.sql.types import StructType, StructField, StringType

aliccp_schema = StructType([
    StructField("click", StringType(), True),
    StructField("purchase", StringType(), True),
    StructField("101", StringType(), True),
    StructField("121", StringType(), True),
    StructField("122", StringType(), True),
    StructField("124", StringType(), True),
    StructField("125", StringType(), True),
    StructField("126", StringType(), True),
    StructField("127", StringType(), True),
    StructField("128", StringType(), True),
    StructField("129", StringType(), True),
    StructField("205", StringType(), True),
    StructField("206", StringType(), True),
    StructField("207", StringType(), True),
    StructField("216", StringType(), True),
    StructField("508", StringType(), True),
    StructField("509", StringType(), True),
    StructField("702", StringType(), True),
    StructField("853", StringType(), True),
    StructField("301", StringType(), True)
])

In [16]:
train_dataset = spark.read.csv(train_path, sep=',', inferSchema=False, header=True, schema=aliccp_schema)
test_dataset = spark.read.csv(test_path, sep=',', inferSchema=False, header=True, schema=aliccp_schema)

In [17]:
train_dataset.show(20)

+-----+--------+-----+---+---+---+---+---+---+---+---+------+----+------+------+----+------+-----+-----+---+
|click|purchase|  101|121|122|124|125|126|127|128|129|   205| 206|   207|   216| 508|   509|  702|  853|301|
+-----+--------+-----+---+---+---+---+---+---+---+---+------+----+------+------+----+------+-----+-----+---+
|    0|       0|24506| 50|  7|  2|  7|  0|  2|  1|  2|     0|6617|241253|     0|3889|     0|    0|    0|  1|
|    0|       0|24506| 50|  7|  2|  7|  0|  2|  1|  2|285875|4932|227985|104091|   0|     0|    0|    0|  1|
|    1|       0|24506| 50|  7|  2|  7|  0|  2|  1|  2|106773|3097| 97188| 64469|5035|     0|14074|  752|  1|
|    0|       0|24506| 50|  7|  2|  7|  0|  2|  1|  2|     0|5971|147495| 87668| 739|     0|    0|    0|  1|
|    0|       0|24506| 50|  7|  2|  7|  0|  2|  1|  2| 45163|3097|220349| 25422|5035|     0|    0|  752|  1|
|    0|       0|24506| 50|  7|  2|  7|  0|  2|  1|  2|     0|6444| 59432| 55177|2915| 34046| 6958|    0|  1|
|    0|       0|245

In [18]:
test_dataset.show(20)

+-----+--------+-----+---+---+---+---+---+---+---+---+------+----+------+-----+----+-----+-----+-----+---+
|click|purchase|  101|121|122|124|125|126|127|128|129|   205| 206|   207|  216| 508|  509|  702|  853|301|
+-----+--------+-----+---+---+---+---+---+---+---+---+------+----+------+-----+----+-----+-----+-----+---+
|    0|       0|24506| 50|  7|  2|  7|  0|  2|  1|  2|     0|2691|121419|92603|2258|69003|46330|    0|  1|
|    0|       0|24506| 50|  7|  2|  7|  0|  2|  1|  2|     0|1913|     0|79854|4074|    0| 1873|18559|  1|
|    0|       0|24506| 50|  7|  2|  7|  0|  2|  1|  2| 60276|2702|140732|15213|   0|    0|    0|    0|  1|
|    0|       0|24506| 50|  7|  2|  7|  0|  2|  1|  2|     0| 230|121419|    0|5787|69003|    0|25901|  1|
|    0|       0|24506| 50|  7|  2|  7|  0|  2|  1|  2|189046|4468| 51249|    0|1480|    0|    0|    0|  1|
|    0|       0|24506| 50|  7|  2|  7|  0|  2|  1|  2| 53047|6272| 36908|90637|   0|    0|    0|    0|  1|
|    0|       0|24506| 50|  7|  2|  7

In [26]:
from pyspark.sql import functions as F
train_dataset_fg = train_dataset.withColumn('label', F.when((F.col('click')=='1')&(F.col('purchase')=='1'), '1').otherwise('0'))
train_dataset_fg = train_dataset_fg.select('label', 
                                            F.col('click').alias('ctr_label'),
                                            F.col('purchase').alias('cvr_label'),
                                            '101','121','122','124','125','126','127','128','129','205','206','207','216','508','509','702','853','301')
train_dataset_fg.show(20)

+-----+---------+---------+-----+---+---+---+---+---+---+---+---+------+----+------+------+----+------+-----+-----+---+
|label|ctr_label|cvr_label|  101|121|122|124|125|126|127|128|129|   205| 206|   207|   216| 508|   509|  702|  853|301|
+-----+---------+---------+-----+---+---+---+---+---+---+---+---+------+----+------+------+----+------+-----+-----+---+
|    0|        0|        0|24506| 50|  7|  2|  7|  0|  2|  1|  2|     0|6617|241253|     0|3889|     0|    0|    0|  1|
|    0|        0|        0|24506| 50|  7|  2|  7|  0|  2|  1|  2|285875|4932|227985|104091|   0|     0|    0|    0|  1|
|    0|        1|        0|24506| 50|  7|  2|  7|  0|  2|  1|  2|106773|3097| 97188| 64469|5035|     0|14074|  752|  1|
|    0|        0|        0|24506| 50|  7|  2|  7|  0|  2|  1|  2|     0|5971|147495| 87668| 739|     0|    0|    0|  1|
|    0|        0|        0|24506| 50|  7|  2|  7|  0|  2|  1|  2| 45163|3097|220349| 25422|5035|     0|    0|  752|  1|
|    0|        0|        0|24506| 50|  7

In [27]:
from pyspark.sql import functions as F
test_dataset_fg = test_dataset.withColumn('label', F.when((F.col('click')=='1')&(F.col('purchase')=='1'), '1').otherwise('0'))
test_dataset_fg = test_dataset_fg.select('label', 
                                         F.col('click').alias('ctr_label'),
                                         F.col('purchase').alias('cvr_label'),
                                         '101','121','122','124','125','126','127','128','129','205','206','207','216','508','509','702','853','301')
test_dataset_fg.show(20)

+-----+---------+---------+-----+---+---+---+---+---+---+---+---+------+----+------+-----+----+-----+-----+-----+---+
|label|ctr_label|cvr_label|  101|121|122|124|125|126|127|128|129|   205| 206|   207|  216| 508|  509|  702|  853|301|
+-----+---------+---------+-----+---+---+---+---+---+---+---+---+------+----+------+-----+----+-----+-----+-----+---+
|    0|        0|        0|24506| 50|  7|  2|  7|  0|  2|  1|  2|     0|2691|121419|92603|2258|69003|46330|    0|  1|
|    0|        0|        0|24506| 50|  7|  2|  7|  0|  2|  1|  2|     0|1913|     0|79854|4074|    0| 1873|18559|  1|
|    0|        0|        0|24506| 50|  7|  2|  7|  0|  2|  1|  2| 60276|2702|140732|15213|   0|    0|    0|    0|  1|
|    0|        0|        0|24506| 50|  7|  2|  7|  0|  2|  1|  2|     0| 230|121419|    0|5787|69003|    0|25901|  1|
|    0|        0|        0|24506| 50|  7|  2|  7|  0|  2|  1|  2|189046|4468| 51249|    0|1480|    0|    0|    0|  1|
|    0|        0|        0|24506| 50|  7|  2|  7|  0|  2

In [28]:
train_dataset_fg.write.parquet(train_output_path, mode="overwrite")

In [29]:
test_dataset_fg.write.parquet(test_output_path, mode="overwrite")

In [30]:
!aws s3 ls s3://dmetasoul-bucket/demo/aliccp/traindata_4000w.parquet/

2022-05-18 04:07:17          0 _SUCCESS
2022-05-18 04:06:45   40365015 part-00000-f7662809-5030-4a59-a7b4-dea6935c329b-c000.snappy.parquet
2022-05-18 04:06:46   40281867 part-00001-f7662809-5030-4a59-a7b4-dea6935c329b-c000.snappy.parquet
2022-05-18 04:06:44   40068487 part-00002-f7662809-5030-4a59-a7b4-dea6935c329b-c000.snappy.parquet
2022-05-18 04:06:44   40675127 part-00003-f7662809-5030-4a59-a7b4-dea6935c329b-c000.snappy.parquet
2022-05-18 04:06:45   40386559 part-00004-f7662809-5030-4a59-a7b4-dea6935c329b-c000.snappy.parquet
2022-05-18 04:06:45   40444874 part-00005-f7662809-5030-4a59-a7b4-dea6935c329b-c000.snappy.parquet
2022-05-18 04:06:46   40602994 part-00006-f7662809-5030-4a59-a7b4-dea6935c329b-c000.snappy.parquet
2022-05-18 04:06:49   40516855 part-00007-f7662809-5030-4a59-a7b4-dea6935c329b-c000.snappy.parquet
2022-05-18 04:07:00   40667213 part-00008-f7662809-5030-4a59-a7b4-dea6935c329b-c000.snappy.parquet
2022-05-18 04:07:00   40648863 part-00009-f7662809-5030-4a59-a7b4-dea

In [31]:
!aws s3 ls s3://dmetasoul-bucket/demo/aliccp/testdata_4000w.parquet/

2022-05-18 04:08:10          0 _SUCCESS
2022-05-18 04:07:33   40941588 part-00000-9c1c191f-bad0-42f3-afd5-b6f41fc724cd-c000.snappy.parquet
2022-05-18 04:07:32   40434223 part-00001-9c1c191f-bad0-42f3-afd5-b6f41fc724cd-c000.snappy.parquet
2022-05-18 04:07:33   40626005 part-00002-9c1c191f-bad0-42f3-afd5-b6f41fc724cd-c000.snappy.parquet
2022-05-18 04:07:32   40628926 part-00003-9c1c191f-bad0-42f3-afd5-b6f41fc724cd-c000.snappy.parquet
2022-05-18 04:07:33   40638544 part-00004-9c1c191f-bad0-42f3-afd5-b6f41fc724cd-c000.snappy.parquet
2022-05-18 04:07:32   40906301 part-00005-9c1c191f-bad0-42f3-afd5-b6f41fc724cd-c000.snappy.parquet
2022-05-18 04:07:32   40905847 part-00006-9c1c191f-bad0-42f3-afd5-b6f41fc724cd-c000.snappy.parquet
2022-05-18 04:07:38   40720216 part-00007-9c1c191f-bad0-42f3-afd5-b6f41fc724cd-c000.snappy.parquet
2022-05-18 04:07:46   40732156 part-00008-9c1c191f-bad0-42f3-afd5-b6f41fc724cd-c000.snappy.parquet
2022-05-18 04:07:48   40798508 part-00009-9c1c191f-bad0-42f3-afd5-b6f

In [32]:
spark.stop()